In [5]:
import pickle

with open("model.pkl", "rb") as file:
    model = pickle.load(file)


In [7]:
import pandas as pd
import numpy as np
import pickle
from sklearn.preprocessing import LabelEncoder
import gradio as gr

try:
    ad = pd.read_csv('click_advertisement.csv')
except FileNotFoundError:
    raise Exception("Dataset 'click_advertisement.csv' not found.")

try:
    with open('model.pkl', 'rb') as file:
        model = pickle.load(file)
except FileNotFoundError:
    raise Exception("Model 'model.pkl' not found.")

ad_topics = sorted(ad['Ad_Topic'].unique())

def preprocess_input(time_spent, age, avg_income, internet_usage, ad_topic, male):
    data = {
        'VistID': [np.random.randint(1000187, 9999708)],
        'Time_Spent': [time_spent],
        'Age': [age],
        'Avg_Income': [avg_income],
        'Internet_Usage': [internet_usage],
        'Ad_Topic': [ad_topic],
        'Country_Name': ['United States'],  
        'City_code': ['City_5'],         
        'Male': [male],
        'Time_Period': ['Morning'],        
        'Weekday': ['Thursday'],          
        'Month': ['January'],              
        'Year': [2020]
    }
    df = pd.DataFrame(data)

   
    le = LabelEncoder()
    df['Male'] = le.fit_transform(df['Male'])

    # One-hot encode categorical variables
    cat_cols = ['Ad_Topic', 'Country_Name', 'City_code', 'Time_Period', 'Weekday', 'Month']
    df_encoded = pd.get_dummies(df, columns=cat_cols)

    # Ensure all expected columns
    expected_cols = model.feature_names_in_
    for col in expected_cols:
        if col not in df_encoded.columns:
            df_encoded[col] = 0
    df_encoded = df_encoded[expected_cols]

    return df_encoded

# Prediction function
def predict_click(time_spent, age, avg_income, internet_usage, ad_topic, male):
    try:
        # Validate inputs
        if not (32.6 <= time_spent <= 91.43 and 19 <= age <= 61 and 13996.5 <= avg_income <= 79484.8 and 104.78 <= internet_usage <= 269.96):
            return "Invalid input values. Check ranges: Time_Spent (32.6–91.43), Age (19–61), Avg_Income (13996.5–79484.8), Internet_Usage (104.78–269.96)."
        
        # Preprocess
        input_data = preprocess_input(time_spent, age, avg_income, internet_usage, ad_topic, male)
        
        # Predict
        pred = model.predict(input_data)[0]
        prob = model.predict_proba(input_data)[0]
        result = "Clicked" if pred == 1 else "Not Clicked"
        confidence = prob[pred] * 100
        
        return f"Prediction: {result}\nConfidence: {confidence:.2f}%"
    except Exception as e:
        return f"Error: {str(e)}"

# Gradio interface
iface = gr.Interface(
    fn=predict_click,
    inputs=[
        gr.Slider(minimum=32.6, maximum=91.43, value=66.85, label="Time Spent (minutes)"),
        gr.Slider(minimum=19, maximum=61, step=1, value=37, label="Age"),
        gr.Slider(minimum=13996.5, maximum=79484.8, value=55930.49, label="Average Income ($)"),
        gr.Slider(minimum=104.78, maximum=269.96, value=184.95, label="Internet Usage (minutes)"),
        gr.Dropdown(choices=ad_topics, value=ad_topics[0], label="Ad Topic"),
        gr.Dropdown(choices=["Yes", "No"], value="No", label="Male")
    ],
    outputs=gr.Textbox(label="Prediction"),
    title="Ad Click Prediction",
    allow_flagging="never"
)

# Launch
iface.launch(inline=True, share=True, quiet=True)

C:\Users\Divyansh Kashyap\AppData\Roaming\Python\Python312\site-packages\gradio\interface.py:415: UserWarning: The `allow_flagging` parameter in `Interface` is deprecated.Use `flagging_mode` instead.
  warnings.warn(


* Running on public URL: https://b61338378ccc104ef0.gradio.live
